In [1]:
!wget https://s3.amazonaws.com/text-datasets/imdb_full.pkl
!wget https://s3.amazonaws.com/text-datasets/imdb_word_index.json

--2019-04-15 15:01:52--  https://s3.amazonaws.com/text-datasets/imdb_full.pkl
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65552540 (63M) [application/octet-stream]
Saving to: ‘imdb_full.pkl’

imdb_full.pkl       100%[===================>]  62.52M  85.2MB/s    in 0.7s    

2019-04-15 15:01:53 (85.2 MB/s) - ‘imdb_full.pkl’ saved [65552540/65552540]

--2019-04-15 15:01:53--  https://s3.amazonaws.com/text-datasets/imdb_word_index.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.139.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.139.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1641221 (1.6M) [application/octet-stream]
Saving to: ‘imdb_word_index.json’

imdb_word_index.jso 100%[===================>]   1.56M  --.-KB/s    in 0.08s   

2019-04-15 15:01:53 (20.3 MB/s) - ‘imdb_word_inde

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle
data = pickle.load(open('imdb_full.pkl', 'rb'))

import json
vocab = json.load(open('imdb_word_index.json'))

In [3]:
inv = {idx:word for word, idx in vocab.items()}

In [4]:
(X, y), (Xt, yt) = data

In [5]:
trainidx = [i for i, x in enumerate(X) if len(x) < 400]
X = [X[i] for i in trainidx]
y = [y[i] for i in trainidx]

testidx = [i for i, x in enumerate(Xt) if len(x) < 400]
testidx, remaining_idx =  train_test_split(testidx, train_size=0.2, random_state=1378)
Xr = [Xt[i] for i in remaining_idx]
yr = [yt[i] for i in remaining_idx]

Xt = [Xt[i] for i in testidx]
yt = [yt[i] for i in testidx]

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
X = [[inv[x] for x in doc] for doc in X]
Xt = [[inv[x] for x in doc] for doc in Xt]
Xr = [[inv[x] for x in doc] for doc in Xr]

In [7]:
X = [" ".join(x) for x in X]
Xt = [" ".join(x) for x in Xt]
Xr = [" ".join(x) for x in Xr]

In [8]:
texts = {'train' : X, 'test' : Xt, 'rem' : Xr}
labels = {'train' : y, 'test' : yt, 'rem' : yr}

In [9]:
import pandas as pd
df_texts = []
df_labels = []
df_exp_splits = []

for key in ['train', 'test', 'rem'] :
    df_texts += texts[key]
    df_labels += labels[key]
    df_exp_splits += [key] * len(texts[key])
    
df = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_splits})
df.to_csv('imdb_dataset.csv', index=False)

In [10]:
%run "../preprocess_data_BC.py" --data_file imdb_dataset.csv --output_file ./vec_imdb.p --word_vectors_type fasttext.simple.300d --min_df 10

Vocabulary size :  14355
Found 13058 words in model out of 14355
